In [22]:
from transformers import AutoTokenizer
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch import nn
import torch.nn as nn
import os
import numpy as np
from transformers import BertForSequenceClassification, AdamW

In [23]:
from transformers import AutoModel
from transformers import BertTokenizer, TFBertModel

model = AutoModel.from_pretrained("google-bert/bert-base-uncased")

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
with open("el_quijote.txt", "r", encoding="utf-8") as file:
    contenido = file.read()



[101, 7570, 2721, 1037, 102]


In [25]:
class Dataset(Dataset):
    def __init__(self, path, tokenizer):
        with open(path, "r", encoding="utf-8") as file:
            self.contenido = file.read()
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.contenido)

    def __getitem__(self, idx):
        sentence = self.x[idx]
        tokens = self.tokenizer(sentence, max_length=512, truncation=True, padding="max_length", return_tensors="pt")
        
        input_ids = tokens["input_ids"].squeeze(0) 
        attention_mask = tokens["attention_mask"].squeeze(0)

        label = torch.tensor(self.y[idx], dtype=torch.long)  # CrossEntropyLoss expects long labels
        
        return input_ids, attention_mask, label

In [31]:
def create_train_test_datasets(filepath, tokenizer, split_ratio=0.8):
    dataset = imbd_dataset(filepath, tokenizer)
    train_size = int(split_ratio * len(dataset))
    test_size = len(dataset) - train_size
    train_data, test_data = random_split(dataset, [train_size, test_size])

    return train_data, test_data
def create_dataloader(training_data, test_data, batch_size):
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

    return train_dataloader, test_dataloader



In [32]:
def select_device():
    device = (
        "cuda"
        if torch.cuda.is_available()
        else "mps"
        if torch.backends.mps.is_available()
        else "cpu"
    )
    print(device)
    return device


In [33]:
class NeuralNetwork(nn.Module):
    def __init__(self, vocab_size, device, embed_size=100, encoders=4, head=10, num_classes=2):
        super(NeuralNetwork, self).__init__()

        # Embedding layer para convertir palabras en vectores densos
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # Capa Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_size, nhead=head, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=encoders)

        # Capa de regularización
        self.dropout = nn.Dropout(0.1)

        # Activación ReLU
        self.relu = nn.ReLU()

        # Capa de salida (embed_size → num_classes)
        self.fc = nn.Linear(embed_size, num_classes)  # Cambié hidden_size a embed_size

    def forward(self, x):
        x = self.embedding(x)  # Convierte tokens en vectores de tamaño embed_size
        x = self.transformer_encoder(x)  # Pasa por el Transformer Encoder
        x = torch.mean(x, dim=1)  # Promediamos los embeddings de la secuencia para clasificación
        x = self.dropout(x)  # Aplicamos dropout
        x = self.relu(x)  # Aplicamos ReLU
        logits = self.fc(x)  # Capa de clasificación final
        return logits


In [37]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 25 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn, device):
    global best_weights, best_correct
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    
    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct/y.size(0)):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    if best_correct < correct:
        best_correct = correct
        best_weights = model.state_dict()


In [38]:
def select_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_model(model, train_dataloader, test_dataloader, loss_fn, optimizer, device, epochs, learning_rate):
    global best_correct, best_weights
    next_train = True
    
    while next_train:
        for t in range(epochs):
            print(f"Epoch {t+1}\n-------------------------------")
            train_loop(train_dataloader, model, loss_fn, optimizer, device)
            best_correct, best_weights = test_loop(test_dataloader, model, loss_fn, device, best_correct, best_weights)

        print("Done! Saving model...")
        torch.save(model.state_dict(), 'model_weights.pth')
        print(f"Best Accuracy: {(100 * best_correct):>0.1f}% Using Learning Rate: {learning_rate}")

        # Preguntar si continuar entrenando
        stop_train = input("Should we continue training? No(Any) Yes(Enter) -> ")
        if stop_train:
            next_train = False
        else:
            # Opción para cambiar el learning rate
            change_lr = input("Change the learning rate? Yes(Any) No(Enter) -> ")
            if change_lr:
                try:
                    learning_rate = float(input("New learning rate: "))
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = learning_rate
                except ValueError:
                    print("Invalid input. Keeping the old learning rate.")

            # Opción para cambiar el número de epochs
            change_epochs = input(f"Change the number of epochs ({epochs})? Yes(Any) No(Enter) -> ")
            if change_epochs:
                try:
                    epochs = int(input("New number of epochs: "))
                except ValueError:
                    print("Invalid input. Keeping the old number of epochs.")

def main(file_path, learning_rate, batch_size, epochs):
    global best_correct, best_weights
    
    loss_fn = nn.CrossEntropyLoss()
    device = select_device()

    # Cargar modelo BERT
    model = AutoModel.from_pretrained("bert-base-uncased")
    model.to(device)

    # Preguntar si entrenar
    train_on = input("Do you want to train? Yes(y), No(enter): ").strip().lower()
    training_data, test_data = create_train_test_datasets(file_path, tokenizer)
    train_dataloader, test_dataloader = create_dataloader(training_data, test_data, batch_size)

    if train_on in ["y", "yes"]:
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        train_model(model, train_dataloader, test_dataloader, loss_fn, optimizer, device, epochs, learning_rate)
    else:
        best_correct, best_weights = test_loop(test_dataloader, model, loss_fn, device, best_correct, best_weights)

if __name__ == '__main__':
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    best_correct = 0
    best_weights = None
    file_path = 'IMDB Dataset.csv'
    learning_rate = 0.0008
    batch_size = 10
    epochs = 10
    main(file_path, learning_rate, batch_size, epochs)

Do you want to train? Yes(y), No(enter):  y


Epoch 1
-------------------------------


AttributeError: 'list' object has no attribute 'squeeze'